In [111]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import os

### Load .npy files containing the rotation and translation matrices

In [109]:
# Load the .npy file
rot_data = np.load("log/modelnet_noisy/test_paper/rotations.npy")
rot_gt = np.load("log/modelnet_noisy/test_paper/rotations_gt.npy")
translation_data = np.load("log/modelnet_noisy/test_paper/translations.npy")
translation_gt = np.load("log/modelnet_noisy/test_paper/translations_gt.npy")
# Print or inspect the data
print(rot_gt[0] - rot_data[0]);

# # Ensure the directory exists
# os.makedirs("transformation/rotation", exist_ok=True)
# # Save DeepGMR rotation matrix
# np.savetxt("transformation/rotation/rotation_0.csv", rot_data[0], delimiter=",")
# # print(data_gt[0])  # Should be a NumPy array

[[-0.04492951  0.01733574 -0.01538122]
 [ 0.01309866 -0.00928712 -0.05094218]
 [ 0.01593968 -0.00631493  0.00330374]]


In [ ]:
# translation_data = np.load("log/modelnet_noisy/test_paper/translations.npy")
# translation_gt = np.load("log/modelnet_noisy/test_paper/translations_gt.npy")

# # Ensure the directory exists
# os.makedirs("transformation/translation", exist_ok=True)
# # Save DeepGMR rotation matrix
# np.savetxt("transformation/translation/translation_0.csv", translation_data[0], delimiter=",")

In [ ]:
# transformation_mat = np.hstack((data[0], translation_reshape))	# Concatenate the rotation and translation matrices	
# transformation_mat = np.vstack((transformation_mat, [0, 0, 0, 1]))	# Add the last row of the transformation matrix
# print(transformation_mat)
# np.savetxt("transformation/transformation_0.txt", transformation_mat, delimiter=" ")

[[ 0.1416055  -0.47654769 -0.86766928 -0.09883341]
 [ 0.89837694 -0.30626968  0.31482884 -0.02494752]
 [-0.41577217 -0.82407552  0.38474998 -0.1051932 ]
 [ 0.          0.          0.          1.        ]]


### Reconstruct transformation matrices from rotation and translation information

In [79]:
def build_transform_mat(rot_data, trans_data):
    # Ensure the directory exists
    os.makedirs("transformation/transformation_matrices", exist_ok=True)
    
    for i in range(len(trans_data)):
        rotation_mat = rot_data[i]
        translation_mat = np.reshape(trans_data[i], (3, 1))
        transformation_mat = np.hstack((rotation_mat, translation_mat))	# Concatenate the rotation and translation matrices	
        transformation_mat = np.vstack((transformation_mat, [0, 0, 0, 1]))	# Add the last row of the transformation matrix
        np.savetxt("transformation/transformation_matrices/" + str(i) + ".txt", transformation_mat, delimiter=" ")  

In [80]:
build_transform_mat(rot_data=rot_data, trans_data=translation_data)

### Import and convert test data from .h5 to .csv compatible with Cloudcompare

In [ ]:
# Load test data from h5 file & check headers
file_path = "data/test/modelnet_noisy.h5"
f = h5py.File(file_path, 'r')
print(list(f.keys()))

['label', 'source', 'target', 'transform']


In [ ]:
# Check shape of data per header
with h5py.File(file_path, 'r') as f:
    print("Labels:", f['label'][:5])  # First 5 labels
    print("Source points shape:", f['source'].shape)  # Shape of source
    print("Target points shape:", f['target'].shape)  # Shape of target
    print("Transform shape:", f['transform'].shape)  # Shape of transform

Labels: [0 0 0 0 0]
Source points shape: (2464, 5000, 3)
Target points shape: (2464, 5000, 3)
Transform shape: (2464, 4, 4)


In [94]:
# Export object to CSV
csv_output_path = "target.csv"

# Open the HDF5 file and read the dataset
with h5py.File(file_path, 'r') as f:
    data = f['target'][0,:]  # Read the full dataset
    print(np.shape(f['target']))

# Convert to a pandas DataFrame
df = pd.DataFrame(data, columns=['X', 'Y', 'Z'])  # Name columns if applicable

# # Save as CSV
# df.to_csv(csv_output_path, index=False)
df.shape
# print(f"Saved CSV to {csv_output_path}")

(2464, 5000, 3)


(5000, 3)

### Extract test objects to CSV:

In [104]:
def extract_object_to_csv(file_path, output_folder, object_type):
    '''
    file_path: str, path to the .h5 file
    Object_type: str, 'source' or 'target'
    filename: str, name of the file to save the data to
    output_path: str, path to the directory to save the file

    output file: output_folder/filename.txt
    '''
    # Ensure the directory exists
    os.makedirs(output_folder, exist_ok=True)

    with h5py.File(file_path, 'r') as f:
        for i in range(len(f[object_type])):
            data = f[object_type][i,:]  # 
            df = pd.DataFrame(data, columns=['X', 'Y', 'Z'])
            df.to_csv(f'{output_folder}/{object_type}_{i}.txt', index = False)



In [106]:
extract_object_to_csv('data/test/modelnet_noisy.h5', 'objects/modelnet_noisy/target', 'target')